# Manual Implementation of a fully connected feed forward Neural Network trained by backpropagation

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import csv
import os
import pandas as pd
from tqdm import tqdm
from random import random
from random import uniform
from csv import reader
from math import exp
from sklearn.metrics import mean_squared_error




## Retrieve and reshape datasets

In [3]:
import os

base_dir = os.path.dirname(os.path.realpath('_file_'))
#dtype='type 'float''

# , dtype=int
data_train = pd.read_csv(base_dir +"/labeled_processed_imgs_x.csv",dtype=float , header=None, delimiter=",").dropna(axis=1, how="all")
train_x = data_train.as_matrix()

#x = np.loadtxt(base_dir +"/Datasets/labeled_processed_imgs.csv", dtype=np.uint8, delimiter=",") # load from text 

data_test = pd.read_csv(base_dir +"/labeled_processed_imgs_test_x.csv.csv",dtype=float , header=None, delimiter=",").dropna(axis=1, how="all")
test_x  = data_test.as_matrix()

data_labels = pd.read_csv(base_dir +"/train_y.csv",dtype=np.uint8 , header=None,delimiter=",").dropna(axis=1, how="all")
train_y  = data_labels.as_matrix()


# Reshape arrays to proper format
train_x = train_x.reshape(-1, 50 * 50) # reshape 
test_x = test_x.reshape(-1, 50 * 50) # reshape 
train_y = train_y.reshape(-1)

#Label training dataset
train = np.concatenate((train_x, train_y[:,None]), axis=1)





## Initialization of the network
n_inputs = Number of inputs for first layer

n_hidden = Number of neurons per hidden layer

n_outputs = Number of outputs of output layer

hidden_layers = Number of hidden layers of the network

In [7]:
start = 0
end = 1

def initialize_network(n_inputs, n_hidden, n_outputs, hidden_layers ):
    network = list()
    for i in range(hidden_layers):
        hidden_layer = list()
        for i in range(n_hidden):
            hidden_layer.append({'weights':[random() for i in range(n_inputs+ 1)]})
        network.append(hidden_layer)
    output_layer = list()
    for i in range(n_outputs):
        output_layer.append({'weights':[random() for i in range(n_hidden+1)]})
    network.append(output_layer)
    return network

## Different activation funtions & respective derivatives


In [9]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1] #bias
    for i in range(len(inputs)):
        activation += weights[i] * inputs[i]
    #print("neuro activa: ", activation)
    return activation


# Transfer neuron activation
def transfer(activation):
    return tanh(activation)

# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return dtanh(output)


def sigmoid(x):
    return 1. / (1 + np.exp(-x))

def dsigmoid(x):
    return x * (1. - x)

def tanh(x):
    return np.tanh(x)

def dtanh(x):
    return 1. - x * x

def ReLU(x):
    return x * (x > 0)

def dReLU(x):
    return 1. * (x > 0)

def softmax(x):
    e = np.exp(x - np.max(x))  # prevent overflow
    if e.ndim == 1:
        return e / np.sum(e, axis=0)
    else:  
        return e / np.array([np.sum(e, axis=1)]).T  # ndim = 2


## Forward propagation

network = Initialized network

row = Unlabeled input image



In [663]:
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        #print("layer \n\n")
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            #print("output: ", neuron['output'])
            new_inputs.append(neuron['output'])
            #if(layer == network[-1]):
                #print(neuron['output'])
        #print(new_inputs[0:10])
        inputs = new_inputs
    return inputs

## Back propagation

network = Initialized network

expected = Labels for last processed image

In [664]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
                
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
            

## Update weights

network = Initialized network

row = Unlabeled input image

l_rate = learning rate


In [665]:
# Update network weights with error
def update_weights(network, row, l_rate):
    inputs = row[:-1]
    for i in range(len(network)):
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                #print("\n old wei: ", neuron['weights'][j], " delta: ", neuron['delta'], " input: ", inputs[j])
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
                #print("new wei: ", neuron['weights'][j])
                
            neuron['weights'][-1] += l_rate * neuron['delta']
            
            


## Train

network = Initialized network

train = Dataset containing the training images

l_rate = learning rate

n_epoch = Number of epochs for training

n_ouputs = Number of classes to identify

In [666]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0        
        for row in train:
            outputs = softmax(forward_propagate(network, row))
            expected = np.zeros(n_outputs)
            expected[int(row[-1])] = 1
            #print(expected)
            #print(outputs)
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(n_outputs)])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.5f, error=%.3f, lastPred=%.5f' % (epoch, l_rate, sum_error, outputs[-1]))

## Predict

network = Initialized network

row = Unlabeled input image


In [667]:
# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

## Split training dataset in several folds 

In [ ]:
n_folds = 200
folds = np.vsplit(train, n_folds)



## Initialize network with corresponding values

In [668]:
n_hidden = 12
hidden_layers = 3
n_inputs = len(train_x[0])
n_outputs = 10 # number of digits 0-9

network = initialize_network(n_inputs, n_hidden, n_outputs,hidden_layers)



## Train  network & Predict some results 

In [706]:
l_rate = 0.8
n_epoch = 50
pred_size=15


for fold in tqdm(range(51,199)):
    train_network(network, folds[fold], l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in tqdm(test_x[0:pred_size]):
        prediction = predict(network, row)
        predictions.append(prediction)
    print(min(predictions), max(predictions), network[-1][min(predictions)])

  0%|          | 0/148 [00:00<?, ?it/s]

>epoch=0, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=1, lrate=0.80000, error=225.090, lastPred=0.10000
>epoch=2, lrate=0.80000, error=225.320, lastPred=0.10822
>epoch=3, lrate=0.80000, error=224.778, lastPred=0.10000
>epoch=4, lrate=0.80000, error=225.022, lastPred=0.10251
>epoch=5, lrate=0.80000, error=225.334, lastPred=0.09999
>epoch=6, lrate=0.80000, error=225.757, lastPred=0.09995
>epoch=7, lrate=0.80000, error=225.314, lastPred=0.10000
>epoch=8, lrate=0.80000, error=225.163, lastPred=0.10000
>epoch=9, lrate=0.80000, error=224.958, lastPred=0.10931
>epoch=10, lrate=0.80000, error=225.060, lastPred=0.09998
>epoch=11, lrate=0.80000, error=225.241, lastPred=0.10000
>epoch=12, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=13, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=14, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=15, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=16, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=17, lrate=0.80000


  1%|          | 1/90 [00:00<00:12,  6.94it/s]

>epoch=49, lrate=0.80000, error=225.092, lastPred=0.10000



  1%|          | 1/148 [04:43<11:33:57, 283.25s/it]

...


  6%|▌         | 5/90 [00:00<00:01, 48.90it/s]

>epoch=49, lrate=0.80000, error=225.000, lastPred=0.10000



 11%|█▏        | 17/148 [1:34:44<12:10:05, 334.39s/it]

4 4 {'output': 0.9999999799496418, 'delta': -4.010067064607316e-08, 'weights': [0.5675673696374056, -0.12067222268781268, 0.36040057666771713, -0.34786998961658955, 0.3765749457021003, 0.09378385814551547, -0.06533134128705373, -0.010307576133322716, -0.5351039654953934, 0.14769638096720433, 2.933058785012793, -1.7902581182842967, 3.288038589516809]}
>epoch=0, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=1, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=2, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=3, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=4, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=5, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=6, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=7, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=8, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=9, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=10, lrate=0.80000, error=225.000, lastPred=0.10000
>epoch=11, lrate=0.

KeyboardInterrupt: 